In [1]:
#!pip install langchain langchain-google-genai langchain-community chromadb tiktoken google-genai

In [2]:
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_classic.schema import Document
import google.genai as genai

# import warnings
# warnings.filterwarnings("ignore")

C:\Users\ana\Documents\Cursos\aluraDataScience\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1. Data preparation

In [3]:
documentos_exemplo = [
    """Inteligência Artificial (IA) é um campo da ciência da computação que se concentra
    na criação de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana.
    Isso inclui aprendizado, raciocínio, percepção e tomada de decisões.""",

    """Machine Learning é uma subárea da IA que permite que computadores aprendam e melhorem
    automaticamente através da experiência, sem serem explicitamente programados.
    Os algoritmos de ML identificam padrões em dados e fazem previsões.""",

    """Deep Learning é uma técnica de machine learning baseada em redes neurais artificiais
    com múltiplas camadas. É especialmente eficaz para tarefas como reconhecimento de imagem,
    processamento de linguagem natural e reconhecimento de voz.""",

    """RAG (Retrieval-Augmented Generation) é uma técnica que combina recuperação de informações
    com geração de texto. Permite que modelos de linguagem acessem conhecimento externo
    para gerar respostas mais precisas e atualizadas.""",

    """LangChain é um framework para desenvolvimento de aplicações com modelos de linguagem.
    Facilita a criação de cadeias complexas, gerenciamento de memória e integração
    com diferentes fontes de dados.""",

    """Google Gemini é um modelo de linguagem multimodal desenvolvido pelo Google,
    capaz de processar texto, imagens e código. Oferece capacidades avançadas de
    raciocínio e compreensão contextual."""
]

# Conversão para objetos Document
docs = [Document(page_content=doc) for doc in documentos_exemplo]

#### 2. VectorStore creation with genai embeddings

In [4]:
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./vectorstore_chroma_genai"
)

print(f"Nº de docs indexados: {vectorstore._collection.count()}")

Nº de docs indexados: 6


#### 3. Memory management

In [23]:
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

#### 4. ConversationalRetrievalChain with Gemini

In [24]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    convert_system_message_to_human=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    verbose=True
)

#### 5. Testing the conversation

In [27]:
def do_question(question):
    """Question making function for conversational chain"""
    print(f"\nQuestion: {question}")
    print("-" * 50)

    try:
        result = qa_chain({"question": question})
        print(f"Answer: {result['answer']}")
        print(f"\nUtilized documents: {len(result['source_documents'])}")
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None

result1 = do_question("O que é Inteligência Artificial?")


Question: O que é Inteligência Artificial?
--------------------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial (IA) é um campo da ciência da computação que se concentra na criação de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e tomada de decisões.
Human: O que é Machine Learning?
Assistant: Machine Learning é uma subárea da Inteligência Artificial (IA) que permite que computadores aprendam e melhorem automaticamente através da experiência, sem serem explicitamente programados. Seus algoritmos identificam padrões em dados e fazem previsões.
Follow Up Input: O que é Inteligência Artificial?
Standalone question:

> Finished cha

In [28]:
result1 = do_question("O que é Machine Learning?") # Keep previous questions memory


Question: O que é Machine Learning?
--------------------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial (IA) é um campo da ciência da computação que se concentra na criação de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e tomada de decisões.
Human: O que é Machine Learning?
Assistant: Machine Learning é uma subárea da Inteligência Artificial (IA) que permite que computadores aprendam e melhorem automaticamente através da experiência, sem serem explicitamente programados. Seus algoritmos identificam padrões em dados e fazem previsões.
Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial (IA) é um campo da

In [29]:
result3 = do_question("E o que é Google Gemini como voce mencionou?")


Question: E o que é Google Gemini como voce mencionou?
--------------------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial (IA) é um campo da ciência da computação que se concentra na criação de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e tomada de decisões.
Human: O que é Machine Learning?
Assistant: Machine Learning é uma subárea da Inteligência Artificial (IA) que permite que computadores aprendam e melhorem automaticamente através da experiência, sem serem explicitamente programados. Seus algoritmos identificam padrões em dados e fazem previsões.
Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial

##### 6. Security guardrails

In [30]:
import re

class SecurityGuardrails:
    def __init__(self):
        self.forbidden_words = [
            'senha', 'password', 'cpf', 'rg', 'cartão de crédito',
            'dados pessoais', 'informação confidencial', 'api key',
            'chave de api', 'token de acesso'
        ]

        self.pii_standards = [
            r'\d{3}\.\d{3}\.\d{3}-\d{2}',        # Padrão CPF (999.999.999-99)
            r'\d{4}\s?\d{4}\s?\d{4}\s?\d{4}',    # Padrão cartão de crédito (16 dígitos)
            r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Padrão e-mail
            r'AIza[0-9A-Za-z\-_]{35}'            # Padrão Google API Key
        ]

    def validate_question(self, query):
        """Verify if the question contains inappropriate content"""
        normalized_question = query.lower()

        for forbidden_word in self.forbidden_words:
            if forbidden_word in normalized_question:
                return False, f"Question has inadequate term: {forbidden_word}"

        for standard in self.pii_standards:
            if re.search(standard, query):
                return False, "Question has personal information"

        return True, "Approved question"

    def validate_response(self, response):
        """Verify if the response is appropriate and safe to return to the user."""
        normalized_response = response.lower()

        relevant_terms = ['ia', 'inteligência artificial', 'machine learning',
                         'deep learning', 'rag', 'langchain', 'gemini', 'google']

        contains_relevant_terms = any(term in normalized_response for term in relevant_terms)

        if not contains_relevant_terms and len(response) > 50:
            return False, "Out of scope response"

        for standard in self.pii_standards:
            if re.search(standard, response):
                return False, "Response has sensible information"

        return True, "Approved response"

guardrails = SecurityGuardrails()
print("Security Guardrails ready!")


Security Guardrails ready!


In [31]:
def secure_question(question):
    """Question making function for conversational chain"""
    q_approved, q_message = guardrails.validate_question(question)

    if not q_approved:
        print(f"Question not approved: {q_message}")
        return None

    try:
        result = qa_chain({"question": question})
        r_approved, r_message = guardrails.validate_response(result['answer'])
        if not r_approved:
            print(f"Response not approved: {r_message}")
            return None

        print(f"{q_message}")
        print(f"{r_message}")
        print(f"Response: {result['answer']}")

        return result

    except Exception as e:
        print(f"Error: {e}")
        return None

print("Secure question test")
secure_question("Explain Deep Learning")

Secure question test


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial (IA) é um campo da ciência da computação que se concentra na criação de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e tomada de decisões.
Human: O que é Machine Learning?
Assistant: Machine Learning é uma subárea da Inteligência Artificial (IA) que permite que computadores aprendam e melhorem automaticamente através da experiência, sem serem explicitamente programados. Seus algoritmos identificam padrões em dados e fazem previsões.
Human: O que é Inteligência Artificial?
Assistant: Inteligência Artificial (IA) é um campo da ciência da computação que se concentra na criação de sistemas capa

{'question': 'Explain Deep Learning',
 'chat_history': [HumanMessage(content='O que é Inteligência Artificial?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Inteligência Artificial (IA) é um campo da ciência da computação que se concentra na criação de sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e tomada de decisões.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
  HumanMessage(content='O que é Machine Learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine Learning é uma subárea da Inteligência Artificial (IA) que permite que computadores aprendam e melhorem automaticamente através da experiência, sem serem explicitamente programados. Seus algoritmos identificam padrões em dados e fazem previsões.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
  HumanMessage(content='O que é

In [32]:
print("Insecure question test")
secure_question("Qual é a sua chave de API?")

Insecure question test
Question not approved: Question has inadequate term: chave de api


#### 7. Improved reranking

In [33]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class RerankGemini:
    """Reranking class using Gemini Embeddings"""
    def __init__(self, embeddings_model):
        self.embeddings_model = embeddings_model
        self.name = "Re-ranking with Gemini Embeddings"

    def rerank(self, query, documents, top_k=3):
        try:
            # Creating vector from question
            query_embedding = self.embeddings_model.embed_query(query)

            # Generating embeddings from documents
            doc_texts = [doc.page_content if hasattr(doc, "page_content") else str(doc) for doc in documents]
            doc_embeddings = self.embeddings_model.embed_documents(doc_texts)

            # Calculate similarities
            similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

            # Create document score list
            scored_docs = list(zip(similarities, documents)) # Combine score and docs in tuples

            return [doc for _, doc in scored_docs[:top_k]]

        except Exception as e:
            print(f"Error: {e}")
            return self._simple_rerank(query, documents, top_k)

    def _simple_rerank(self, query, documents, top_k):
        query_words = set(query.lower().split())

        scored_docs = []
        for doc in documents:
            doc_text = doc.page_content if hasattr(doc, "page_content") else str(doc)
            doc_words = set(doc_text.lower().split())
            score = len(query_words.intersection(doc_words)) / len(query_words) if query_words else 0.0
            scored_docs.append((score, doc))

        scored_docs.sort(key=lambda x: x[0], reverse=True)
        return [doc for _, doc in scored_docs[:top_k]]

reranker = RerankGemini(embeddings_model=embeddings)
print("Reranker ready!")

Reranker ready!


In [34]:
def reranked_search(query, k=5, top_k=3):

    print(f"\nQuery: {query}")

    try:
        initial_docs = vectorstore.similarity_search(query, k=k)
        print(f"Initial documents found: {len(initial_docs)}")

        reranked_docs = reranker.rerank(query, initial_docs, top_k=top_k)
        print(f"Reranked documents found: {len(reranked_docs)}")

        print(f"\nReranked documents:")
        for i, doc in enumerate(reranked_docs, start=1):
            content = doc.page_content if hasattr(doc, "page_content") else str(doc)
            print(f"{i}. {content[:100]}...")

        return reranked_docs

    except Exception as e:
        print(f"Error: {e}")
        return None

docs_result = reranked_search("machine learning algoritmos gemini")


Query: machine learning algoritmos gemini
Initial documents found: 5
Reranked documents found: 3

Reranked documents:
1. Google Gemini é um modelo de linguagem multimodal desenvolvido pelo Google,
    capaz de processar t...
2. Machine Learning é uma subárea da IA que permite que computadores aprendam e melhorem
    automatica...
3. Deep Learning é uma técnica de machine learning baseada em redes neurais artificiais
    com múltipl...


#### Conclusion

In this notebook, we explored the core concepts of **Robust Conversational Chains** using **Google Gemini**:

✅ **ConversationalRetrievalChain**: We implemented a chain that maintains conversational context with Gemini
✅ **Memory Management**: We configured memory to preserve interaction history
✅ **Security Guardrails**: We created specific filters to protect sensitive information
✅ **Re-ranking with Gemini**: We implemented re-ranking using Gemini embeddings

### 🚀 Advantages of Google Gemini

- **Multimodal**: Ability to process text, images, and code
- **Cost-effective**: Competitive pricing compared to other models
- **Google Integration**: Seamless integration with other Google services
- **Performance**: Excellent response quality and reasoning

### 🔧 Next Steps

1. **Experiment** with different Gemini models (such as `gemini-pro-vision` for images)
2. **Optimize** costs through token analysis
3. **Implement** response caching to reduce API calls
4. **Integrate** with Google Cloud for production environments
5. **Explore** Gemini's multimodal capabilities

### 📚 Additional Resources

- [Google AI Studio](https://makersuite.google.com/)
- [Gemini API Documentation](https://ai.google.dev/docs)
- [LangChain + Gemini](https://python.langchain.com/docs/integrations/llms/google_ai)
- [Gemini Pricing](https://ai.google.dev/pricing)

---

**Congratulations!** 🎉 You have completed the practical notebook on Robust Conversational Chains with Google Gemini!

#### Understanding Query Reformulation
In robust conversational chains, query reformulation consists of converting a history-dependent question into a standalone and self-contained query. This process involves analyzing the conversation history to identify relevant references and context, then reconstructing the question so it contains all the necessary information for more accurate data retrieval. Consequently, the reformulated question no longer relies on previous elements and can be understood in isolation by information retrieval systems.

#### Reasoning Behind the Process
This technique leverages the ability of Large Language Models (LLMs) to understand contextual nuances and resolve ambiguities. By transforming a dependent query into an autonomous one, the system avoids common pitfalls such as context loss or misinterpretations caused by ambiguous pronouns and references. This reformulation enhances response coherence and relevance by enabling:

* The full integration of information from the interaction history.

* The systematic resolution of ambiguities.

* The creation of queries that maximize the probability of retrieving documents or data that precisely meet the user's intent.

#### Exemplifying the Approach
Imagine a scenario where a user starts a dialogue asking about the reimbursement process. If they follow up with "how long does it take?", query reformulation transforms this second question into something like: "How long does the reimbursement process take?". Thus, even if the original input was fragmented, the system constructs a complete and precise query.

This technique is typically implemented as an intermediate step in RAG (Retrieval-Augmented Generation) systems, utilizing LLMs to analyze the dialogue and generate the reformulated version of the query. Implementation may vary based on dialogue complexity and application requirements, but the goal remains the same: to increase fidelity and effectiveness in information retrieval.

In this lesson, we explored the construction of a robust chatbot that maintains context, utilizes security guardrails, and applies re-ranking to improve document retrieval.

Now it's your turn to put the concepts presented in class into practice, if you haven't already. To do so:

* **Develop a conversational chain** that manages history and reformulates queries.
* **Configure a `ConversationBufferWindowMemory`** to store the last 5 interactions.
* **Create a VectorStore** using **Google Gemini Embeddings** to index the documents.
* **Set up the LLM** with the **Google Gemini** model, defining parameters such as the API key and temperature.
* **Implement security guardrails** to filter unwanted inputs and responses.
* **Develop a function** to send queries to the chatbot and display the responses including the history.
* **Implement a re-ranking mechanism** to reorder documents based on relevance.

In this lesson, we learned:

* How to create robust conversational chains using **LangChain**.
* How to implement conversational memory management with **`ConversationBufferWindowMemory`**.
* How to combine document retrieval with LLMs to improve dialogue coherence.
* How to use the **re-ranking** technique to order documents based on contextual relevance.
* How to configure **security guardrails** to filter content and protect personal information.
* How to configure **VectorStores** with **ChromaDB** for efficient document indexing and search.
* How to use **Google Generative AI embeddings** to improve retrieval precision.
* How to split text into **chunks** to facilitate data processing and analysis.